<a href="https://colab.research.google.com/github/crisbpadilla/CapstoneProject-1-M2M/blob/main/CapstoneProject_2_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine learning- compressive strenght prediction modeling
  In the present capston project, the same data used in the first project will be scaled and transform to preprocess the data, prepared the training and testing data and evaluate the different regression models that could be used to predict the compressive strength of the concrete mix.

  So, the dataset was obtain from kaggle:

In [201]:
import pandas as pd
import numpy as np

concrete=pd.read_csv('concrete.csv')

In the first capston project it was possible to identify that the main components that helps to improve the development and higher values of compressive strength were the SCM and the superplastice.
So due this, we will firstly checked the correlation matrix between the variables to have  a better idea of it and see if this could help to take desicions in the future models.

In [202]:
from bokeh.plotting import figure, output_notebook, show, output_notebook
from bokeh.models import ColumnDataSource, LinearColorMapper,ColorBar

output_notebook()
material_corr=concrete.corr()

material_corr = material_corr.stack().reset_index()
material_corr.columns = ['x', 'y', 'value']

material_corr['value'] = material_corr['value'].round(2)

source = ColumnDataSource(material_corr)

# Define color mapper
color_mapper = LinearColorMapper(palette="Inferno256", low=-1, high=1)

# Create figure
p = figure(title="Correlation Heatmap", x_range=list(concrete.columns), y_range=list(concrete.columns),
           x_axis_location="above", width=600, height=600,
           tools="hover", tooltips="@x, @y: @value{0.2f}")

p.rect(x="x", y="y", width=1, height=1, source=source,
       fill_color={'field': 'value', 'transform': color_mapper}, line_color=None)

# Add text annotations
from bokeh.models import LabelSet
labels = LabelSet(x='x', y='y', text='value', source=source,text_align='center',
                  text_baseline='middle',text_font_size='8pt',text_color='white')
p.add_layout(labels)

# Add color bar
color_bar = ColorBar(color_mapper=color_mapper, label_standoff=12, border_line_color=None, location=(0, 0))
p.add_layout(color_bar, 'right')

p.xaxis.major_label_orientation = 0.75

# Show plot
show(p)


Reviewing the heatmap it is possible to observe the inverse correlation between water an superplastic, variable that could be flattened as they are both lightly correlated ( superplastic is used to speed up the compressive strength gain and the reduction of water in the mix).
 So we will check the performance of the model, reducing the demionsionality of this model , taking water and superplastic into one unique component.

# Preprocessing data

In [203]:
print(concrete.head())
print(concrete.info())

   cement   slag    ash  water  superplastic  coarseagg  fineagg  age  \
0   141.3  212.0    0.0  203.5           0.0      971.8    748.5   28   
1   168.9   42.2  124.3  158.3          10.8     1080.8    796.2   14   
2   250.0    0.0   95.7  187.4           5.5      956.9    861.2   28   
3   266.0  114.0    0.0  228.0           0.0      932.0    670.0   28   
4   154.8  183.4    0.0  193.3           9.1     1047.4    696.7   28   

   strength  
0     29.89  
1     23.51  
2     29.22  
3     45.85  
4     18.29  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1030 entries, 0 to 1029
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   cement        1030 non-null   float64
 1   slag          1030 non-null   float64
 2   ash           1030 non-null   float64
 3   water         1030 non-null   float64
 4   superplastic  1030 non-null   float64
 5   coarseagg     1030 non-null   float64
 6   fineagg       1030 non

the dataframe is build by the raw materials that composed the concrete mix  (kg/m^3), age (days) and compressive strength of the sample.

with the idea of see the impact of adding   categorical values to the model, depending the type of mix, the next code includes the next categories.

    * all mix
    * slag
    * ash
    * superplastic
    * slag-ash
    * slag-superplastic
    * ash-superplastic.
    * just cement


In [204]:
concrete_pluscat=concrete.copy()
# cuales son los escenarios? 9 (matriz 3x3) , la mitad menos 1 ~ 3
#a) slag-ash b)slag-superplastic c)ash-superplastic

concrete_pluscat['type_mix'] = 'all mix'
concrete_pluscat.loc[
    (concrete_pluscat['slag']>0) & (concrete_pluscat[['superplastic','ash']] == 0).all(axis=1),
    'type_mix',
] = 'slag'
concrete_pluscat.loc[
    (concrete_pluscat['ash']>0) & (concrete_pluscat[['superplastic','slag']] == 0).all(axis=1),
    'type_mix',
] = 'ash'
concrete_pluscat.loc[
    (concrete_pluscat['superplastic']>0) & (concrete_pluscat[['slag','ash']] == 0).all(axis=1),
    'type_mix',
] = 'superplastic'

concrete_pluscat.loc[
    (concrete_pluscat[['slag', 'ash']].gt(0).all(axis=1)) & (concrete_pluscat['superplastic'] == 0),
    'type_mix',
] = 'slag-ash'

concrete_pluscat.loc[
    (concrete_pluscat[['slag', 'superplastic']].gt(0).all(axis=1)) & (concrete_pluscat['ash'] == 0),
    'type_mix',
] = 'slag-superplastic'

concrete_pluscat.loc[
    (concrete_pluscat[['ash', 'superplastic']].gt(0).all(axis=1)) & (concrete_pluscat['slag'] == 0),
    'type_mix',
] = 'ash-superplastic'
concrete_pluscat.loc[
    (concrete_pluscat['cement']>0) & (concrete_pluscat[['superplastic','slag','ash']] == 0).all(axis=1),
    'type_mix',
] = 'just cement'

In [205]:
concrete_pluscat.groupby('type_mix')['strength'].count()

,strength
type_mix,
all mix,225
ash,6
ash-superplastic,233
just cement,209
slag,164
slag-superplastic,170
superplastic,23


there is no slag-ash mix. giving 6 dummie columns dropping the first one.

The dummies for the categorical values were created so we can added to the model later

In [206]:
# Create the dataframe with all the numerical and categorical values
concrete_data=pd.get_dummies(data=concrete_pluscat,columns=['type_mix'],drop_first=True).astype(int)
concrete_data.head()


,cement,slag,ash,water,superplastic,coarseagg,fineagg,age,strength,type_mix_ash,type_mix_ash-superplastic,type_mix_just cement,type_mix_slag,type_mix_slag-superplastic,type_mix_superplastic
0,141,212,0,203,0,971,748,28,29,0,0,0,1,0,0
1,168,42,124,158,10,1080,796,14,23,0,0,0,0,0,0
2,250,0,95,187,5,956,861,28,29,0,1,0,0,0,0
3,266,114,0,228,0,932,670,28,45,0,0,0,1,0,0
4,154,183,0,193,9,1047,696,28,18,0,0,0,0,1,0


In [207]:
RED = "\033[31m"
RESET = "\033[0m"

ptg_data28=(len(concrete[concrete['age']<=100])/len(concrete))*100


print(f'The percentage of data under 100 days over the total is {RED}{round(ptg_data28,3)} %{RESET}')

print(f"{RED}{len(concrete[concrete['age']>100])}{RESET} samples are over 100 days")
print(f"{RED}{len(concrete[concrete['age']<=100])}{RESET} samples are under 100 days")

The percentage of data under 100 days over the total is 93.981 %
62 samples are over 100 days
968 samples are under 100 days


in the first capstone project, it is possible to identify that big amount of the data is below 100 days, using mathematical operations we can observe that ***93,981 %*** of the samples are obtain before the 100 days, so this data is the most representative of the whole dataset, being the samples above that age potential noise in the model as they also represent values of just two particular mixes (slag and all mix). Cause of this, the samples above 100 days will be removed of the model an this will predict compressive strength between 0 and 100 days.

In [208]:
# concrete_model_data=concrete_data[concrete_data['age']<=100]
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

concrete_model_data=concrete_data[concrete_data['age']<=100]
X_100=concrete_model_data.drop(columns=['strength'])
y_100=concrete_model_data['strength']
X_train_100,X_test_100,y_train_100,y_test_100=train_test_split(X_100,y_100,test_size=0.25)
print(y_train_100.shape,X_train_100.shape,y_test_100.shape)

(726,) (726, 14) (242,)


In [209]:


X=concrete_data.drop(columns=['strength'])
y=concrete_data['strength']

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.25)


In [210]:
# Seprating my numeric variables and nominal/categorical values
num_col=[a for a in concrete.columns]
num_col.remove('strength')
#numerical values
X_train_num=X_train[num_col]
X_test_num=X_test[num_col]
print(X_train_num.shape)
#numerical values dataset 2 (100 days)
X_train_num_100=X_train_100[num_col]
X_test_num_100=X_test_100[num_col]
print(X_train_num_100.shape)

#nominal values
X_train_nom=X_train.drop(columns=num_col)
X_test_nom=X_test.drop(columns=num_col)
#nominal values dataset 2 (100 days)
X_train_nom_100=X_train_100.drop(columns=num_col)
X_test_nom_100=X_test_100.drop(columns=num_col)

print(X_train.shape)

(772, 8)
(726, 8)
(772, 14)


##Scalling the data

In [211]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler

scale_data=StandardScaler()
print(pd.DataFrame(scale_data.fit_transform(X_train_num)))

X_train_num_scale=scale_data.fit_transform(X_train_num)
X_test_num_scale=scale_data.fit_transform(X_test_num)

#dataset 2 (100 days)
X_train_num_scale_100=scale_data.fit_transform(X_train_num_100)
X_test_num_scale_100=scale_data.fit_transform(X_test_num_100)


            0         1         2         3         4         5         6  \
0    0.469852  0.778003 -0.843605  2.133612 -1.020923 -0.509796 -2.185067   
1    0.754611  1.321601 -0.843605 -0.771652  0.823297 -0.355357 -0.215266   
2    0.469852  0.778003 -0.843605  2.133612 -1.020923 -0.509796 -2.185067   
3    0.640707 -0.864358 -0.843605  0.998743 -1.020923  0.030741  0.029430   
4   -0.289505 -0.864358 -0.843605  0.181638 -1.020923  1.793922  0.139543   
..        ...       ...       ...       ...       ...       ...       ...   
767 -0.849530  2.489759 -0.843605  0.181638 -1.020923  0.339620 -0.839240   
768  1.371588 -0.320760 -0.843605  2.133612 -1.020923 -0.509796 -2.185067   
769 -1.039369 -0.286062  1.851555 -0.771652 -0.014985  0.442579  0.249656   
770 -0.659691  0.269102 -0.469710  0.000059 -0.014985  1.201905  0.151778   
771 -0.612231 -0.239798  1.072607 -1.861126  0.823297  1.330605  0.237421   

            7  
0    0.679044  
1   -0.617491  
2    4.974790  
3   -0.5081

###Lineal model

In [212]:
from sklearn import linear_model
#using all data
lineal=linear_model.LinearRegression()
lineal.fit(X_train_num_scale,y_train)
l0=round(lineal.score(X_train_num_scale,y_train),3)
l1=round(lineal.score(X_test_num_scale,y_test),3)
score_lineal=[l0,l1]
#using data with age below 100

lineal_100=linear_model.LinearRegression()
lineal_100.fit(X_train_num_scale_100,y_train_100)
la=round(lineal_100.score(X_train_num_scale_100,y_train_100),3)
lb=round(lineal_100.score(X_test_num_scale_100,y_test_100),3)
score_lineal_100=[la,lb]
df_lineal=pd.DataFrame({'type_score':['Training','Testing'],'all data R^2':score_lineal,'filtered data R^2':score_lineal_100}).set_index('type_score')
print('values of scores for lineal model')
print(df_lineal)

values of scores for lineal model
            all data R^2  filtered data R^2
type_score                                 
Training           0.613              0.757
Testing            0.611              0.771


###Reciprocal logaritmic


In [213]:
import numpy as np

def addReciprocalLogFeatures(numeric):
    log_feats = numeric.copy()
    valid = (log_feats != 1) & (log_feats > 0)
    log_feats[valid] = np.log(log_feats[valid]) / np.log(10)
    log_feats[log_feats <= 0] = 1e-10
    rec_log_feats = log_feats
    return np.hstack([numeric, rec_log_feats, numeric * rec_log_feats])

X_train_reclog = addReciprocalLogFeatures(X_train_num_scale)
X_test_reclog = addReciprocalLogFeatures(X_test_num_scale)
#for filtered data
X_train_reclog_100 = addReciprocalLogFeatures(X_train_num_scale_100)
X_test_reclog_100 = addReciprocalLogFeatures(X_test_num_scale_100)


In [214]:
#log regression
lineal_rl=linear_model.LinearRegression()
lineal_rl.fit(X_train_reclog,y_train)
lr0=lineal_rl.score(X_train_reclog,y_train)
lr1=lineal_rl.score(X_test_reclog,y_test)
score_lr=[lr0,lr1]
# for filtered data
lineal_rl_100=linear_model.LinearRegression()
lineal_rl_100.fit(X_train_reclog_100,y_train_100)
lr0_100=lineal_rl_100.score(X_train_reclog_100,y_train_100)
lr1_100=lineal_rl_100.score(X_test_reclog_100,y_test_100)
score_lr_100=[lr0_100,lr1_100]
predicted_reclog_100=lineal_rl_100.predict(X_test_reclog_100)

df_lr=pd.DataFrame({'type_score':['Training','Testing'],'all data R^2':score_lr,'filtered data R^2':score_lr_100}).set_index('type_score')
print('values of scores for lineal model')
print(df_lr)

values of scores for lineal model
            all data R^2  filtered data R^2
type_score                                 
Training        0.794094           0.848194
Testing         0.748726           0.813409


In [215]:
BLUE = "\033[1;34m"
RESET = "\033[0m"

df1 = pd.DataFrame({"Predicted_100": predicted_reclog_100, "Actual_100": y_test_100})
df1['% Difference(100 days)'] = (abs(df1['Predicted_100']-df1['Actual_100'])/df1['Actual_100'])*100

print("Percentage Difference between Predicted and Actual Values (rec.log model with age bellow 100 days)")
print(df1.head())
print(f"\nMean % Difference between Predicted and Actual Values of data bellow 100 days:{BLUE}{str(round(df1['% Difference(100 days)'].mean(),3))}%{RESET}")

Percentage Difference between Predicted and Actual Values (rec.log model with age bellow 100 days)
     Predicted_100  Actual_100  % Difference(100 days)
967      58.783368          71               17.206524
527      61.704438          65                5.070096
347      41.078554          50               17.842892
351      20.025880          17               17.799293
61        6.081116          10               39.188838

Mean % Difference between Predicted and Actual Values of data bellow 100 days:19.256%


Linear and reciprocal logaritmic model, shown higher score in the testing samples than the training samples. which is not a good signal ....

###cubic model

In [216]:
#lineal regession gives a r2 too low and the function seems to fit to a log function
from sklearn.preprocessing import PolynomialFeatures,FunctionTransformer

from sklearn.pipeline import make_pipeline

square = PolynomialFeatures(degree=3, include_bias=False)
X_train_scale_cubic = square.fit_transform(X_train_num_scale)
X_test_scale_cubic = square.fit_transform(X_test_num_scale)

reg=linear_model.LinearRegression()
reg.fit(X_train_scale_cubic,y_train)
prediction_train=reg.predict(X_train_scale_cubic)
prediction_test=reg.predict(X_test_scale_cubic)

C0=reg.score(X_train_scale_cubic,y_train)
C1=reg.score(X_test_scale_cubic,y_test)
cubic_score=[C0,C1]

#filtered data
cubic_f=PolynomialFeatures(degree=3, include_bias=False)
X_train_scale_cubic_100 = cubic_f.fit_transform(X_train_num_scale_100)
X_test_scale_cubic_100 = cubic_f.fit_transform(X_test_num_scale_100)

C0_100=reg.score(X_train_scale_cubic_100,y_train_100)
C1_100=reg.score(X_test_scale_cubic_100,y_test_100)
cubic_score_100=[C0_100,C1_100]
Predict_cubic=reg.predict(X_test_scale_cubic)

df_cubic=pd.DataFrame({'type_score':['Training','Testing'],'All data R^2':cubic_score,'Filtered data R^2':cubic_score_100}).set_index('type_score')
print('values of scores for cubic model')
print(df_cubic)

values of scores for cubic model
            All data R^2  Filtered data R^2
type_score                                 
Training        0.931029          -0.600592
Testing         0.794541          -1.001459


In [217]:
df_cubic = pd.DataFrame({"Predicted": Predict_cubic, "Actual": y_test})
df_cubic['% Difference(all data)'] = (abs(df_cubic['Predicted']-df_cubic['Actual'])/df_cubic['Actual'])*100

print("Percentage Difference between Predicted and Actual Values (cubic model)")
print(df_cubic.head())
print(f"\nMean % Difference between Predicted and Actual Values with all data:{BLUE}{str(round(df_cubic['% Difference(all data)'].mean(),3))} %{RESET}")


Percentage Difference between Predicted and Actual Values (cubic model)
     Predicted  Actual  % Difference(all data)
985  40.559368      32               26.748026
945  34.653861      44               21.241224
574  68.747848      55               24.996087
824  65.421072      67                2.356610
934  39.999831      33               21.211609

Mean % Difference between Predicted and Actual Values with all data:21.744 %


**Removing the data above 100 days improve the model consideraby  the scores in all 3 different models but the cubic polynomial one, which shows very negative values.**

## ***Adding nominal values***
  The two best models were selected and added the nominal values, so in this way will be possible to asses their performance.
  
  * Cubic model with all data
  * rec.log model with filtered data

In [218]:

X_train_ln_full = np.hstack([X_train_scale_cubic, X_train_nom])
X_test_ln_full = np.hstack([X_test_scale_cubic, X_test_nom])
#filtered data
X_train_ln_full_100 = np.hstack([X_train_reclog_100, X_train_nom_100])
X_test_ln_full_100 = np.hstack([X_test_reclog_100, X_test_nom_100])
X_train_ln_full.shape

(772, 170)

In [219]:
RED = "\033[1;31m"
GREEN= "\033[1;32m"
RESET = "\033[0m"

ln_model= linear_model.RidgeCV()
ln_model.fit(X_train_ln_full,y_train)
train_predict_ln=ln_model.predict(X_train_ln_full)
test_predict_ln=ln_model.predict(X_test_ln_full)
print('Considering the nominal values for the two best models in their respective scenarios')
print('Cubic model with all the data')
print(f'For the train data,the R^2 for the {RED}cubic model{RESET} is : {round(ln_model.score(X_train_ln_full,y_train),3)}')
print(f'For the test data,the R^2 for the {RED}cubic model{RESET}is : {round(ln_model.score(X_test_ln_full,y_test),3)}')

#filtered model

ln_model_100= linear_model.RidgeCV()
ln_model_100.fit(X_train_ln_full_100,y_train_100)
train_predict_ln_100=ln_model_100.predict(X_train_ln_full_100)
test_predict_ln_100=ln_model_100.predict(X_test_ln_full_100)

print('logaritmic model filtered (bellow 11 days)')
print(f'For the train data,the R^2 for the {GREEN}reciprocal log model{RESET} is : {round(ln_model_100.score(X_train_ln_full_100,y_train_100),3)}')
print(f'For the test data,the R^2 for the {GREEN}reciprocal log model{RESET} is : {round(ln_model_100.score(X_test_ln_full_100,y_test_100),3)}')

Considering the nominal values for the two best models in their respective scenarios
Cubic model with all the data
For the train data,the R^2 for the cubic model is : 0.933
For the test data,the R^2 for the cubic modelis : 0.8
logaritmic model filtered (bellow 11 days)
For the train data,the R^2 for the reciprocal log model is : 0.851
For the test data,the R^2 for the reciprocal log model is : 0.81


In [220]:
df0 = pd.DataFrame({"Predicted": test_predict_ln, "Actual": y_test})
df1 = pd.DataFrame({"Predicted_100": test_predict_ln_100, "Actual_100": y_test_100})
df0['% Difference(all data)'] = (abs(df0['Predicted']-df0['Actual'])/df0['Actual'])*100
df1['% Difference(100 days)'] = (abs(df1['Predicted_100']-df1['Actual_100'])/df1['Actual_100'])*100

print("Percentage Difference between Predicted and Actual Values (cubic model)")
print(df0.head())
print("Percentage Difference between Predicted and Actual Values (rec.log model with age bellow 100 days)")
print(df1.head())
print(f"\nMean % Difference between Predicted and Actual Values with all data:{RED}{str(round(df0['% Difference(all data)'].mean(),3))} %{RESET}")
print(f"\nMean % Difference between Predicted and Actual Values of data bellow 100 days:{GREEN}{str(round(df1['% Difference(100 days)'].mean(),3))}%{RESET}")

Percentage Difference between Predicted and Actual Values (cubic model)
     Predicted  Actual  % Difference(all data)
985  41.445004      32               29.515637
945  36.815962      44               16.327360
574  65.145256      55               18.445920
824  64.217166      67                4.153483
934  38.338727      33               16.177961
Percentage Difference between Predicted and Actual Values (rec.log model with age bellow 100 days)
     Predicted_100  Actual_100  % Difference(100 days)
967      58.014126          71               18.289964
527      65.308949          65                0.475306
347      43.089657          50               13.820687
351      20.019443          17               17.761429
61        9.220618          10                7.793824

Mean % Difference between Predicted and Actual Values with all data:21.906 %

Mean % Difference between Predicted and Actual Values of data bellow 100 days:19.552%


It is possible to observe that 2 of the 3 models improved narrowing the data to just 968 samples , being the cubic model negativly affected by this reduction on the data but clearly showing signs of overfitting in the predictive model as the difference between the R^2 for train and test data is above 5% difference in most of the cases, reaching in some of them 10%, however with the reciprocal logaritmic model, the reduction in the data gives us  slightly lower R^2 for the test data but a difference between the coeficient of determination in between 1 and 2%.


## applying PCA

In [221]:
X_train_WS=X_train[['water','superplastic']]
X_train_WS_scaled= scale_data.fit_transform(X_train_WS)
X_test_WS=X_test[['water','superplastic']]

X_test_WS_scaled= scale_data.fit_transform(X_test_WS)
print(X_train_WS_scaled.shape,X_test_WS_scaled.shape)

pca=PCA(n_components=1)
X_train_pca=pca.fit_transform(X_train_WS_scaled)
X_test_pca=pca.fit_transform(X_test_WS_scaled)

# # train and test data without water and superplastic

col=[a for a in concrete.columns]
items_to_remove = ['strength','water','superplastic']
col = [c for c in col if c not in items_to_remove]

X_train_non_PCA=X_train[col]
X_test_non_PCA=X_test[col]
#scaled data
X_train_non_PCA_scale=scale_data.fit_transform(X_train_non_PCA)
X_test_non_PCA_scale=scale_data.fit_transform(X_test_non_PCA)

X_train_PCA_scaled=np.hstack([X_train_pca,X_train_non_PCA_scale])
X_test_PCA_scaled=np.hstack([X_test_pca,X_test_non_PCA_scale])


(772, 2) (258, 2)


In [222]:
X_train_reclog_PCA = addReciprocalLogFeatures(X_train_PCA_scaled)
X_test_reclog_PCA = addReciprocalLogFeatures(X_test_PCA_scaled)

reg=linear_model.LinearRegression()
reg.fit(X_train_reclog_PCA,y_train)
prediction_train=reg.predict(X_train_reclog_PCA)
prediction_test=reg.predict(X_test_reclog_PCA)

C0=reg.score(X_train_reclog_PCA,y_train)
C1=reg.score(X_test_reclog_PCA,y_test)
PCA_score=[C0,C1]
df_PCA=pd.DataFrame({'type_score':['Training','Testing'],'All data R^2':PCA_score}).set_index('type_score')
print('values of scores for PCA model')
print(df_PCA)


values of scores for PCA model
            All data R^2
type_score              
Training        0.780772
Testing         0.746873


In this case the R^2 for the train data was reduced but the R^2 for the test data increased in relation with the previous model.
 So, now left to add the nominal variabes to verify the score with this model.

In [223]:

X_train_ln_full = np.hstack([X_train_reclog_PCA, X_train_nom])
X_test_ln_full = np.hstack([X_test_reclog_PCA, X_test_nom])


In [224]:
ln_model= linear_model.RidgeCV()
ln_model.fit(X_train_ln_full,y_train)
train_predict_pca=ln_model.predict(X_train_ln_full)
test_predict_pca=ln_model.predict(X_test_ln_full)
print(ln_model.score(X_train_ln_full,y_train))
print(ln_model.score(X_test_ln_full,y_test))

0.7867237244691684
0.750037615052764


In [225]:
df = pd.DataFrame({"Predicted": test_predict_pca, "Actual": y_test})
df['% Difference'] = (abs(df['Predicted']-df['Actual'])/df['Actual'])*100

print("Percentage Difference between Predicted and Actual Values (cubic model)")
print(df.head())
print("\nMean % Difference between Predicted and Actual Values: " + str(round(df['% Difference'].mean(),3)) +"%")

Percentage Difference between Predicted and Actual Values (cubic model)
     Predicted  Actual  % Difference
985  43.649422      32     36.404444
945  48.397638      44      9.994632
574  63.459292      55     15.380531
824  48.980279      67     26.895105
934  32.445589      33      1.680034

Mean % Difference between Predicted and Actual Values: 25.516%
